<img src="Pictures/RedfishLogo.png" alt="Redfish Logo" style="width: 250px;"/> 

Version 0.35

# OData queries

## Introduction

This Jupyter notebook demonstrates several OData queris (`$expand`, `only`...) using [Bash](https://www.gnu.org/software/bash/) and the [cURL](https://curl.haxx.se/) tool against an HPE iLO 5. For didactic reasons, commands presented in this notebook may be not optimized and don't follow the recommended [best practises](https://developer.hpe.com/blog/getting-started-with-the-redfish-api-part-2).

**[OData](https://www.odata.org/)** is an open protocol to allow the creation and consumption of queryable and interoperable RESTful APIs in a simple and standard way.

More details are in the [HPE Redfish API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#introduction).



### `$expand` query parameter

![ExpandSupport](Pictures/ExpandSupport.png)

### `only` Query parameter

![OnlyQueryParamter](Pictures/OnlyQueryParameter.png)






## Create environment variables and session

The following `bash` code defines environment variables (i.e. IP address, username, password....) depending on your student ID number stored in variable `$Stud`. It creates as well several `.json` files containing various HTTP workloads required to POST or PATCH the managed iLO. 

In [ ]:
# Create BMC related variables
iLO5_IP=ilo-etc1
#iLO5_IP=172.16.50.99
iLO5_URI="https://${iLO5_IP}"

# iLO 5 Administrator credentials
iLO5_User="student"
iLO5_Passwd='P@ssw0rd!'

# Minimum required Redfish headers
HeaderODataVersion="OData-Version: 4.0"
HeaderContentType="Content-Type: application/json"

# Data files
ResponseHeaders="ResponseHeaders.txt"             # Used to hold HTTP response headers
SessionData="./CreateSession-data.json"           # Body/Workload used to create the Redfish session

cat > ${SessionData} << __EOF__
{
        "UserName": "$iLO5_User",
        "Password": "$iLO5_Passwd"
}
__EOF__


# Verify we can reach the remote Bmc
curl --insecure --silent \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --request GET ${iLO5_URI}/redfish | jq  || echo "WARNING: Problem reaching the remote BMC"
  

## Create the Redfish session

Redfish allows basic authentication and session authentication. With basic authentication you need to supply the required credentials at each and every HTTP request. Session oriented authentication is achieved by requesting a `Token` that will be sent in the headers of all requests until the removal of the session.

To get this `Token`, POST a session request with the remote BMC credentials in its body. The `Token` as well as the `session location` will be in the headers of the response.

In [ ]:
echo 'Create iLO 5 Session' 

curl --dump-header  $ResponseHeaders \
     --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --request POST --data "@$SessionData" \
     ${iLO5_URI}/redfish/v1/SessionService/Sessions > /dev/null 2>&1
     
Token=$(awk '/X-Auth-Token/ {print $NF}' $ResponseHeaders | tr -d '\r')
SessionLocation="$iLO5_URI"$(awk '/^Loca.*Se/ {gsub("https://.*/red", "/red", $NF);print $NF}' $ResponseHeaders | tr -d '\r')

echo "Token: $Token"
echo -e "Session Location: $SessionLocation\n"

### Retrieve iLO 5 supported OData protocol features

The following command retrieves the OData supported protocol features.

In [ ]:
echo "Retrieve supported OData features:"
curl --insecure --silent --noproxy "localhost, 127.0.0.1"  \
     --header  "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}/redfish/v1 | jq '.ProtocolFeaturesSupported'


###  GET collection without `$expand` 


In [ ]:
echo "GET FirmwareInventory collection"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}/redfish/v1/UpdateService/FirmwareInventory | jq 
       

###  Expand collection with `$expand` 


In [ ]:
echo "Retrieve ALL Firmware versions with a single GET"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/UpdateService/FirmwareInventory?$expand=.' | \
       jq '.Members | .[] | {Name: .Name, Description: .Description, Version: .Version}'

## Expand collection only if it contains one element

### Without the `only` query parameter

In [ ]:
echo "Chassis collection"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/Chassis' | jq

### With the `only` query parameter


In [ ]:
echo "Chassis collection"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request GET ${iLO5_URI}'/redfish/v1/Chassis?only' | jq

## More OData queries

In addition to the `$expand` and `only` OData queries Redfish implementations can support others like `$filter`, `$top` and `$skip`. 

### `$filter` query

Extract IML logs created after a specific date.

In [ ]:
Filter="filter=Created+gt+'2020-04-01T08:27:03Z'"

curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}/redfish/v1/Systems/1/LogServices/IML/Entries'?$'$Filter | jq

### `$top` query 

Extract the oldest Log records within all possible iLO log files.

In [ ]:
# 
ODataQuery='?$top=1'

# Retrieve LogURIs locations
LogURIs=$(curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}/redfish/v1/Systems/1/LogServices | jq -r '.Members | .[] | ."@odata.id"')

# For each LogURI in LogURIs, retrieve the location of entries.
LogEntriesURIs=""
  for LogURI in $LogURIs ; do
    LogEntriesURI=$(curl --insecure --silent \
         --header "$HeaderContentType" --header "$HeaderODataVersion" \
         --header "X-Auth-Token: $Token" \
         --request GET ${iLO5_URI}$LogURI | jq -r '.Entries | ."@odata.id"')

    LogEntriesURIs="${LogEntriesURI} $LogEntriesURIs"
done

# Remove trailing space if any
LogEntriesURIs=$(echo ${LogEntriesURIs%%*()})
echo -e "Oldest Log Entries URIs: ${iLO5_URI}${LogEntriesURIs}${ODataQuery}\n"


curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}${LogEntriesURIs}${ODataQuery} | jq


### `$skip` query

Skip the first 75 log entries and print the rest.

In [ ]:


curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
--request GET ${iLO5_URI}'/redfish/v1/Systems/1/LogServices/IML/Entries?$skip=75' | jq

## Delete sessions

It is extremely important to delete Redfish sessions to avoid reaching the maximum number of opened sessions in a BMC, preventing any access to it. Read this [article](https://developer.hpe.com/blog/managing-ilo-sessions-with-redfish) for more detail.

In [ ]:
echo "Body response of a session deletion:"

curl --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --header "X-Auth-Token: $Token" \
     --request DELETE $SessionLocation | jq

## Wrap up

In this notebook you performed the following actions:

  * Create and delete a token based Redfish session
  * Retrieved iLO 5 supported OData protocol features
  * Expand a collection with OData `$expand` query operator
  * Expand a collection with OData `only` query operator
  * Filter log entries with `$filter`, `$top` and `$skip` queries
  * Delete a Redfish session